<a href="https://colab.research.google.com/github/solomontessema/building-ai-agents/blob/main/notebooks/2.2.%20Implementing_a_Runnable_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building and Composing Runnables with LangChain Expression Language (LCEL)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/solomontessema/Agentic-AI-with-Python/blob/main/notebooks/Building%20with%20LangChain/Implementing_a_Runnable_Sequence.ipynb)

## Learning Objectives

By the end of this notebook, you will:

1. Understand what Runnables are and why they replaced legacy Chains
2. Master the pipe operator (`|`) for composing processing pipelines
3. Learn how to capture and inspect intermediate results
4. Build multi-step reasoning workflows using LCEL
5. Use `RunnablePassthrough` and `RunnableLambda` for advanced composition

---

## Part 1: Understanding Runnables vs. Legacy Chains

### What Changed in LangChain?

LangChain evolved from using **Class-based Chains** (like `LLMChain`, `SimpleSequentialChain`) to **Runnables** with **LangChain Expression Language (LCEL)**.

#### Legacy Approach (Deprecated):
```python
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run({"topic": "AI"})
```

#### Modern Approach (LCEL):
```python
chain = prompt | llm | StrOutputParser()
result = chain.invoke({"topic": "AI"})
```

### Why Runnables?

| Feature | Legacy Chains | Runnables (LCEL) |
|---------|---------------|------------------|
| **Composition** | Verbose class instantiation | Clean pipe operator `\|` |
| **Streaming** | Limited/manual | Built-in `.stream()` |
| **Debugging** | Harder to trace | Better introspection |
| **Status** | **Deprecated** | **Current Standard** |

### Core Runnable Methods

Every Runnable implements these methods:

- **`invoke(input)`** - Run synchronously and return final result
- **`stream(input)`** - Stream outputs as they're generated
---

## Part 2: Setup and Installation

In [ ]:
# Install required packages
!pip install -q langchain-openai langchain-core python-dotenv

In [ ]:
# Import required libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Verify API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY in your .env file")

print("✅ Setup complete!")

## Part 3: Building Your First Runnable

### The Simplest Runnable: Prompt → LLM → Parser

This is the fundamental pattern in LCEL:

In [ ]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "What are the latest trends in {topic}?"
)

# Compose the chain using the pipe operator
chain = prompt | llm | StrOutputParser()

# Invoke the chain
result = chain.invoke({"topic": "machine learning"})
print(result)

### What Just Happened?

1. **`prompt`** - Formats the input dict into a message for the LLM
2. **`|`** - The pipe operator passes output to the next step
3. **`llm`** - Sends the formatted message to OpenAI's API
4. **`StrOutputParser()`** - Extracts the text content from the LLM response

The beauty of LCEL is that each component is **composable** - you can rearrange, add, or remove steps easily.

---

## Part 4: Multi-Step Reasoning Pipelines

### Example: Summarize → Rephrase

Let's build a two-step pipeline where:
1. First step summarizes input text
2. Second step rephrases the summary professionally

In [ ]:
# Step 1: Summarize input text
summarize_prompt = ChatPromptTemplate.from_template(
    "Summarize this content concisely: {input_text}"
)
summarize_chain = summarize_prompt | llm | StrOutputParser()

# Step 2: Rephrase the summary
rephrase_prompt = ChatPromptTemplate.from_template(
    "Rephrase this summary in a professional tone: {text}"
)
rephrase_chain = rephrase_prompt | llm | StrOutputParser()

print("✅ Individual chains created")

### Connecting the Chains

  

In [ ]:
def print_intermediate(x):
  print("Intermediate Result: " , x)
  return x

# Compose pipeline with proper input mapping
pipeline_v1 = (
    summarize_chain
    | RunnableLambda(print_intermediate)
    | rephrase_chain
)

# Test it
input_text = "The global AI market is projected to grow significantly over the next decade."
result = pipeline_v1.invoke({"input_text": input_text})

print("Final Result:")
print(result)

## Part 5: Advanced Multi-Step Workflow

### Use Case: Extract Entities → Summarize → Format

Let's build a three-step pipeline:

In [ ]:
# Step 1: Extract named entities
entity_prompt = ChatPromptTemplate.from_template(
    "Extract all people, organizations, and locations from: {input_text}\n"
    "Return as a simple list."
)
entity_chain = entity_prompt | llm | StrOutputParser()

# Step 2: Summarize entities
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize these extracted entities in one sentence: {entities}"
)
summary_chain = summary_prompt | llm | StrOutputParser()

# Step 3: Format as a paragraph
format_prompt = ChatPromptTemplate.from_template(
    "Format this into a user-friendly paragraph: {summary}"
)
format_chain = format_prompt | llm | StrOutputParser()

# Compose the full pipeline
full_pipeline = (
    entity_chain
    | summary_chain
    | format_chain
)

# Test it
test_input = {
    "input_text": "OpenAI and Google DeepMind are competing in developing AGI. "
                  "Sam Altman and Demis Hassabis are leading their respective organizations."
}

result = full_pipeline.invoke(test_input)
print("Final Formatted Output:")
print(result)

## Part 6: Parallel Execution with `RunnableParallel`

Sometimes you want to run multiple analyses in parallel and combine results:

In [ ]:
# Create parallel analysis chains
sentiment_prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment (positive/negative/neutral) of: {text}"
)
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

keywords_prompt = ChatPromptTemplate.from_template(
    "Extract 3-5 key keywords from: {text}"
)
keywords_chain = keywords_prompt | llm | StrOutputParser()

language_prompt = ChatPromptTemplate.from_template(
    "Classify the language style (formal/informal/technical) of: {text}"
)
language_chain = language_prompt | llm | StrOutputParser()

# Run all three in parallel
parallel_chain = RunnableParallel(
    sentiment=sentiment_chain,
    keywords=keywords_chain,
    language=language_chain
)

# Test it
analysis_result = parallel_chain.invoke({
    "text": "The quarterly earnings exceeded expectations, demonstrating robust growth "
            "in our core business segments."
})

print("Parallel Analysis Results:\n")
print(f"Sentiment: {analysis_result['sentiment']}")
print(f"Keywords: {analysis_result['keywords']}")
print(f"Language Style: {analysis_result['language']}")

### 🚀 Performance Note

When using `RunnableParallel`, all chains execute **concurrently**. This means:
- Faster execution (all API calls happen at once)
- More efficient use of resources
- Better for independent analyses

---

## Part 7: Streaming Results

One major advantage of LCEL is built-in streaming support:

In [ ]:
# Create a chain for streaming
streaming_prompt = ChatPromptTemplate.from_template(
    "Write a short story about {topic} in 3 paragraphs."
)
streaming_chain = streaming_prompt | llm | StrOutputParser()

# Stream the output
print("Streaming story about AI...\n")
for chunk in streaming_chain.stream({"topic": "a friendly AI robot"}):
    print(chunk, end="", flush=True)

print("\n\n✅ Streaming complete!")

### Why Streaming Matters

- **Better UX**: Users see results immediately instead of waiting
- **Faster perceived performance**: First token latency vs. full completion
- **Long-form content**: Essential for essays, reports, stories

---